# EVALUACIÓN FINAL MÓDULO 2 
Este script realiza el proceso completo:
1. Fase 1: Extracción de 100 registros desde API y limpieza de DataFrame.
2. Fase 2: Creación de la base de datos 'adalab_pelis_json' mediante Python.
3. Fase 3: Inserción de datos.
4. Fase 4: Consultas.

In [1]:
# Requisito previo: Instalación de librerías necesarias
# pip install requests / mysql-connector-python / pandas

import requests 
import mysql.connector 
import pandas as pd

## FASE 1: Extracción y transformación

In [2]:
url_pelis_adalab = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

peticion = requests.get(url_pelis_adalab) # Petición a la API

In [3]:
if peticion.status_code == 200: # Verificamos si la conexión ha sido exitosa (status 200 = ok)
        print("Conexión API exitosa.")
        datos_iniciales = peticion.json()
else: # Si algo falla, que nos devuelva el código para buscar la explicación del fallo
        print(f"Error al conectar con la API: {peticion.status_code}")

Conexión API exitosa.


In [4]:
if len(datos_iniciales) > 0: # Vemos cuántos registros tenemos y las claves del primero para saber qué podemos encontrar en la bbdd
        print(f"Registros totales detectados: {len(datos_iniciales)}")
        print("Claves del primer registro:", datos_iniciales[0].keys())


Registros totales detectados: 100
Claves del primer registro: dict_keys(['id', 'titulo', 'año', 'duracion', 'genero', 'adultos', 'subtitulos'])


In [5]:
# Aunque nos piden extraer 100 registros, que son los que hay, limitamos datos para evitar sobrecarga si la API crece
limite = 100 
datos_limitados = datos_iniciales[:limite] # Cogiendo del inicio al limite marcado

In [6]:
display(datos_limitados) # Visualización rápida de los datos

[{'id': 1,
  'titulo': 'The Godfather',
  'año': 1972,
  'duracion': 175,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 2,
  'titulo': 'The Godfather Part II',
  'año': 1974,
  'duracion': 202,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 3,
  'titulo': 'Pulp Fiction',
  'año': 1994,
  'duracion': 154,
  'genero': 'Crimen',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 4,
  'titulo': 'Forrest Gump',
  'año': 1994,
  'duracion': 142,
  'genero': 'Drama',
  'adultos': False,
  'subtitulos': ['es', 'en', 'fr']},
 {'id': 5,
  'titulo': 'The Dark Knight',
  'año': 2008,
  'duracion': 152,
  'genero': 'Acción',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 6,
  'titulo': 'Fight Club',
  'año': 1999,
  'duracion': 139,
  'genero': 'Drama',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 7,
  'titulo': 'Inception',
  'año': 2010,
  'duracion': 148,
  'genero': 'Ciencia ficción',
  'adultos

In [7]:
lista_peliculas = [] # Creamos una lista vacía y vamos añadiendo diccionarios completos (filas) en cada iteración.

for peli in datos_limitados: 
        nueva_peli = {
            'Titulo': peli.get('titulo', 'desconocido'), # Si no hay título, ponemos 'Desconocido' para evitar vacíos
            'Año': peli.get('año', 0), # Si no hay año, ponemos 0 para evitar vacíos
            'Duracion': peli.get('duracion', 0),
            'Genero': peli.get('genero', 'sin clasificar'),
            'Contenido adulto': peli.get('adultos', False), # Tipo de dato booleano
            'Subtitulos': peli.get('subtitulos',[])
         }
        lista_peliculas.append(nueva_peli)  # Añadimos la película procesada a la lista

In [8]:
df_peliculas = pd.DataFrame(lista_peliculas)
print("DataFrame creado correctamente.")
pd.DataFrame(lista_peliculas) 

DataFrame creado correctamente.


,Titulo,Año,Duracion,Genero,Contenido adulto,Subtitulos
0,The Godfather,1972,175,Crimen,False,"[es, en]"
1,The Godfather Part II,1974,202,Crimen,False,"[es, en]"
2,Pulp Fiction,1994,154,Crimen,True,"[es, en]"
3,Forrest Gump,1994,142,Drama,False,"[es, en, fr]"
4,The Dark Knight,2008,152,Acción,False,"[es, en]"
...,...,...,...,...,...,...
95,La vita è bella,1997,116,Drama,False,"[es, en, it]"
96,Requiem for a Dream,2000,102,Drama,True,"[es, en]"
97,Memento,2000,113,Thriller,True,"[es, en]"
98,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False,"[es, en]"


In [9]:
# AJUSTES VISUALES : 
df_peliculas.index = range(1, len(df_peliculas) + 1) # Índice desde 1 (no desde 0)
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.width', 1000)        # Ancho máximo para evitar saltos de línea
pd.set_option('display.colheader_justify', 'center') # Centrar encabezados
print(df_peliculas)
    

                    Titulo                   Año  Duracion   Genero   Contenido adulto   Subtitulos 
1                            The Godfather  1972     175      Crimen        False           [es, en]
2                    The Godfather Part II  1974     202      Crimen        False           [es, en]
3                             Pulp Fiction  1994     154      Crimen         True           [es, en]
4                             Forrest Gump  1994     142       Drama        False       [es, en, fr]
5                          The Dark Knight  2008     152      Acción        False           [es, en]
..                                     ...   ...       ...       ...               ...           ...
96                         La vita è bella  1997     116       Drama        False       [es, en, it]
97                     Requiem for a Dream  2000     102       Drama         True           [es, en]
98                                 Memento  2000     113    Thriller         True          

## FASE 2: Creación de la base de datos

In [18]:
# Hacemos la conexión con el servidor
try:
# IMPORTANTE: Definir credenciales antes de ejecutar.
    cnx = mysql.connector.connect(
        user='root',
        password='AlumnaAdalab', # Dejar password='' para entornos locales sin contraseña, en caso de error consultar si en el equipo tiene MySQL contraseña y ponerla
        host='127.0.0.1', # Nos conectamos al servidor de mysql, sin especificar la database porque la vamos a crear aquí
        auth_plugin = 'mysql_native_password'
    )
    print("¡Conexión exitosa a MySQL!")

except mysql.connector.Error as err:
    # Si es un error de acceso denegado (ej. contraseña o usuario incorrecto)
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Algo está mal con tu nombre de usuario o contraseña.")
    # Si la base de datos no existe
    # Para cualquier otro tipo de error
    else:
        print(err) 
        print("Código de Error:", err.errno) 
        print("SQLSTATE", err.sqlstate) 
        print("Mensaje", err.msg) 

¡Conexión exitosa a MySQL!


In [19]:
mycursor = cnx.cursor() 

mycursor.execute("CREATE SCHEMA IF NOT EXISTS pelis_adalab;") # Creamos la base de datos
mycursor.execute("USE pelis_adalab;") 


In [20]:
# Para evitar duplicados y problemas en las tablas, dejamos un borrado de tablas para poder ejecutar todo el código correctamente
mycursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
mycursor.execute("DROP TABLE IF EXISTS peliculas_idiomas;")
mycursor.execute("DROP TABLE IF EXISTS peliculas;")
mycursor.execute("DROP TABLE IF EXISTS idiomas;")
mycursor.execute("SET FOREIGN_KEY_CHECKS = 1;")

In [21]:
mycursor = cnx.cursor() 

In [22]:
# Creación de tablas: 1º tabla principal con las películas
mycursor.execute("USE pelis_adalab;") 
mycursor.execute("""
    CREATE TABLE  IF NOT EXISTS peliculas (
        id_pelicula INT AUTO_INCREMENT PRIMARY KEY,
        titulo VARCHAR(255) NOT NULL,
        fecha YEAR, 
        duracion INT,
        genero VARCHAR(100),
        adulto BOOLEAN
    );
""")
# En vez de año, utilizamos 'fecha' para evitar problemas con la ñ

In [23]:
# Creación de 2º tabla: idiomas para poder realizar subconsultas y joins
mycursor.execute("""
    CREATE TABLE IF NOT EXISTS idiomas (
        id_idioma INT AUTO_INCREMENT PRIMARY KEY,
        idioma VARCHAR(50) UNIQUE
    );
""")

In [24]:
# Creación de la 3º tabla: con claves foráneas y la que establece referencias. Relación de n a m porque muchas películas pueden tener varios idiomas en sus subtitulos y vs.
mycursor.execute("""
    CREATE TABLE IF NOT EXISTS peliculas_idiomas (
        id_pelicula INT,
        id_idioma INT,
        PRIMARY KEY (id_pelicula, id_idioma),
        FOREIGN KEY (id_pelicula) REFERENCES peliculas(id_pelicula),
        FOREIGN KEY (id_idioma) REFERENCES idiomas(id_idioma)
    );
""")

## FASE 3: Inserción de datos

In [25]:
mycursor.execute("USE pelis_adalab;") 
for contador, (index, fila) in enumerate (df_peliculas.iterrows()): # Recorreremos el DataFrame fila por fila con un contador limpio pra la generación de IDs
    id_seguro = contador + 1
    titulo_peli = fila['Titulo'] # Preparamos los datos en variables simples para que sea más fácil de leer
    fecha_peli = fila['Año']
    duracion_peli = fila['Duracion']
    genero_peli = fila['Genero']
    es_adulto = fila['Contenido adulto']
             
    mycursor.execute( # Insertamos la película usando esas variables
        "INSERT INTO peliculas (id_pelicula, titulo, fecha, duracion, genero, adulto) VALUES (%s, %s, %s, %s, %s, %s)",
        (id_seguro, titulo_peli, fecha_peli, duracion_peli, genero_peli, es_adulto)
    )
            
    id_actual_peli = id_seguro # Guardamos el ID que MySQL le ha dado a esta película

    lista_de_subtitulos = fila['Subtitulos'] # Sacamos la lista de subtítulos a una variable      
    # Como 'lista_de_subtitulos' ya es una lista ['es', 'en'], la recorremos directamente
    for iso_code in lista_de_subtitulos:
    # Insertamos el idioma individualmente
        codigo_individual = str(iso_code).strip() # Buena práctica para limpiar posibles espacios
        if codigo_individual:
            mycursor.execute("SELECT id_idioma FROM idiomas WHERE idioma = %s", (codigo_individual,))
            res_idioma = mycursor.fetchone() # Recuperamos el primer registro encontrado por el SELECT

            if res_idioma: # Si existe, recuperamos el ID existente
                id_actual_idioma = res_idioma[0]
            else: # Si no existe, lo insertamos de nuevo usando la columna 'idioma'
                mycursor.execute("INSERT INTO idiomas (idioma) VALUES (%s)", (codigo_individual,))
                mycursor.execute("SELECT id_idioma FROM idiomas WHERE idioma = %s", (codigo_individual,))
                res_nuevo = mycursor.fetchone()

                if res_nuevo:
                    id_actual_idioma = res_nuevo[0]
                else:
                     # PLAN B (Fallback): Si el SELECT falla por sincronización, usamos lastrowid
                    # Esto evita el error "NoneType is not subscriptable"
                    id_actual_idioma = mycursor.lastrowid

            mycursor.execute( # Con el id_idioma creado autoincremental, creamos la tabla intermedia 'peliculas_idiomas'
                    "INSERT IGNORE INTO peliculas_idiomas (id_pelicula, id_idioma) VALUES (%s, %s)",
                    (id_actual_peli, id_actual_idioma)
                )
cnx.commit() # De vital importancia para que los cambios se apliquen a la bbdd
print("Datos guardados con éxito")
exito = True

Datos guardados con éxito


In [26]:
if cnx is not None and cnx.is_connected():
    cnx.close() # Cerramos conexión para la realización de la siguiente consulta